# Evaluate models

In [3]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
import views_dataviz
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *
# Other packages
import pickle as pkl
import os

# sklearn

from sklearn.metrics import mean_squared_error

In [4]:
# Common parameters:

dev_id = 'Fatalities002'
run_id = 'Fatalities002'
EndOfHistory = 508
RunGeneticAlgo = False
level = 'cm'

username = os.getlogin()

steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

fi_steps = [1,3,6,12,36]
# Specifying partitions

calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{username}/Dropbox (ViEWS)/ViEWS/'
localpath = f'/Users/{username}/Pickles/'
overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/ViEWS predicting fatalities/Tables/'



print('Dropbox path set to',Mydropbox)
print('Overleaf path set to',overleafpath)
print('Local path set to',localpath)

Dropbox path set to /Users/havardhegre1/Dropbox (ViEWS)/ViEWS/
Overleaf path set to /Users/havardhegre1/Dropbox (ViEWS)/Apps/Overleaf/ViEWS predicting fatalities/Tables/
Local path set to /Users/havardhegre1/Pickles/


In [5]:
def FatalitiesHistory(Fatality_cutoff=10,Time_cutoff=12):
    ''' Function to retrieve from viewser a dataframe with dependent variable and a characterisation of recent conflict history  '''

    # initialise QS
    history_colname = "ts_ged_sb_f" + str(Fatality_cutoff) + "_t" + str(Time_cutoff)

    queryset = Queryset("fatalities_history", "country_month")
        # target variable
    queryset = queryset.with_column(Column("ln_ged_sb_dep", from_table = "ged2_cm", from_column = "ged_sb_best_sum_nokgi")
                     .transform.ops.ln()
                     .transform.missing.fill()
                    )   
    queryset = queryset.with_column(Column(history_colname, from_table = "ged2_cm", from_column = "ged_sb_best_sum_nokgi")             
                 .transform.missing.replace_na()
                 .transform.bool.gte(Fatality_cutoff)
                 .transform.temporal.time_since()
                 .transform.missing.replace_na()
                 .transform.bool.gte(Time_cutoff)
                                   )
    history_df = queryset.publish().fetch()
    return history_df, history_colname


def CreateEvaluationDf(stored_modelname_test,Fatality_cutoff=10,Time_cutoff=12,NumberOfMonths=48):
    test_df = pd.DataFrame.forecasts.read_store(stored_modelname_test, run=run_id)
    test_df.replace([np.inf, -np.inf], 0, inplace=True) 
    history_df,history_colname = FatalitiesHistory(Fatality_cutoff,Time_cutoff)
    # NOTE: assumption is that panels are balanced!
    NumberOfMonths = 48
    for step in steps:
        colname = history_colname + '_s' + str(step)
        fromdate = test_partitioner_dict['predict'][0] - step
        todate = test_partitioner_dict['predict'][1] - step
        test_df[colname] = history_df[history_colname].loc[fromdate:todate]
    return test_df,history_colname

In [6]:
EnsembleList = []
genetic = {
        'modelname': 'ensemble_genetic',
        'algorithm': '',
        'depvar': "ln_ged_sb_dep",
        'predstore_calib': 'cm_ensemble_genetic_calib',
        'predstore_test': 'cm_ensemble_genetic_test'   
    }    

EnsembleList.append(genetic)

In [7]:
for model in EnsembleList:
    stored_modelname_test = model['predstore_test']
    ensemble_test_df,history_colname = CreateEvaluationDf(stored_modelname_test,Fatality_cutoff=10,Time_cutoff=12,NumberOfMonths=48)

pr_46_cm_ensemble_genetic_test.parquet
 .      o      O  

In [8]:
def EvaluateModel(df,colname):
#    print(df[colname].value_counts())
    grouped = df.groupby(colname)

    Evaluation_results = [] # list to hold evaluation results

    stepcols = ['ln_ged_sb_dep']
    for step in steps:
        col = 'step_pred_' + str(step)
        mse_test_all = mean_squared_error(df[col], df['ln_ged_sb_dep'])
        colname = history_colname + '_s' + str(step)
#        print(df[colname].value_counts())
        grouped_dfs = df.groupby(colname)
        percentiles = (0.25,0.5,0.75,0.90,0.95,0.98,0.99,0.995)
        for name, group in grouped_dfs:
            if name == 1:
                mse_test_lowconflict = mean_squared_error(group[col], group['ln_ged_sb_dep'])
    #            print('0', name, mse_lowconflict)
            if name == 0:
                mse_test_highconflict = mean_squared_error(group[col], group['ln_ged_sb_dep'])
    #            print(name)
    #    print(col, mse_test_all, mse_lowconflict, mse_highconflict)
        Results = {
            'MSE_all':  mse_test_all,
            'RMSE_all': np.sqrt(mse_test_all),
            'MSE_lowconflict':  mse_test_lowconflict,
            'RMSE_lowconflict': np.sqrt(mse_test_lowconflict),
            'MSE_highconflict':  mse_test_highconflict,
            'RMSE_highconflict': np.sqrt(mse_test_highconflict),
        }
        Evaluation_results.append(Results)

    Evaluation_results_df = pd.DataFrame(Evaluation_results)
    return Evaluation_results_df

colname = history_colname + '_s3'
df = ensemble_test_df
Evaluation_ensemble_df = EvaluateModel(df,colname)


In [9]:
colname = history_colname + '_s3'
print(ensemble_test_df[colname].value_counts())
grouped = ensemble_test_df.groupby(colname)
percentiles = (0.25,0.5,0.75,0.90,0.95,0.98,0.99,0.995)
for name, group in grouped:
    print(name)
    print(group['ln_ged_sb_dep'].describe(percentiles = percentiles))

1.0    7142
0.0    1453
Name: ts_ged_sb_f10_t12_s3, dtype: int64
0.0
count    1453.000000
mean        2.800732
std         2.105010
min         0.000000
25%         0.693147
50%         2.944439
75%         4.189655
90%         5.341375
95%         6.885427
98%         7.497091
99%         7.786520
99.5%       7.916671
max         8.227643
Name: ln_ged_sb_dep, dtype: float64
1.0
count    7142.000000
mean        0.031606
std         0.237822
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
90%         0.000000
95%         0.000000
98%         0.693147
99%         1.386294
99.5%       1.791759
max         4.997212
Name: ln_ged_sb_dep, dtype: float64


In [10]:
Evaluation_ensemble_df

,MSE_all,RMSE_all,MSE_lowconflict,RMSE_lowconflict,MSE_highconflict,RMSE_highconflict
0,0.263019,0.512854,0.045369,0.213000,1.320488,1.149125
1,0.336829,0.580370,0.051922,0.227863,1.666023,1.290745
2,0.392120,0.626195,0.055702,0.236013,1.931525,1.389793
3,0.500948,0.707777,0.054181,0.232769,2.539762,1.593663
4,0.476096,0.689997,0.056691,0.238100,2.345559,1.531522
5,0.558336,0.747219,0.061020,0.247022,2.857804,1.690504
6,0.539356,0.734409,0.062961,0.250920,2.794076,1.671549
7,0.555954,0.745623,0.071846,0.268041,2.961212,1.720817
8,0.644472,0.802790,0.084873,0.291329,3.498103,1.870322
9,0.565435,0.751954,0.083016,0.288124,3.019693,1.737726


In [11]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels(level)
evaluation_allmodels = Evaluation_ensemble_df[['MSE_all','MSE_lowconflict','MSE_highconflict']]
    
i = 0
for model in ModelList:
    print(i, model['modelname'], model['data_train'])
    stored_modelname_test = model['predstore_test']
    model['test_df'],model['history_colname'] = CreateEvaluationDf(stored_modelname_test,Fatality_cutoff=10,Time_cutoff=12,NumberOfMonths=48)
    colname = model['history_colname'] + '_s3'
    model['Evaluation_results_df'] = EvaluateModel(model['test_df'],colname)
    cn_all = 'MSE_all_' + model['modelname']
    evaluation_allmodels[cn_all] = model['Evaluation_results_df']['MSE_all']
    cn_lc = 'MSE_lowconflict_' + model['modelname']
    evaluation_allmodels[cn_lc] = model['Evaluation_results_df']['MSE_lowconflict']
    cn_hc = 'MSE_highconflict_' + model['modelname']
    evaluation_allmodels[cn_hc] = model['Evaluation_results_df']['MSE_highconflict']
    i = i + 1

0 fatalities002_baseline_rf baseline002
pr_46_cm_fatalities002_baseline_rf_test.parquet
 .    

/var/folders/bz/1cx7kmbj2919xmdqxy83mbs80000gp/T/ipykernel_95901/3966367593.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_allmodels[cn_all] = model['Evaluation_results_df']['MSE_all']
/var/folders/bz/1cx7kmbj2919xmdqxy83mbs80000gp/T/ipykernel_95901/3966367593.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_allmodels[cn_lc] = model['Evaluation_results_df']['MSE_lowconflict']


1 fatalities002_conflicthistory_rf conflict_ln
pr_46_cm_fatalities002_conflicthistory_rf_test.parquet
 .    2 fatalities002_conflicthistory_gbm conflict_ln
pr_46_cm_fatalities002_conflicthistory_gbm_test.parquet
 .    3 fatalities002_conflicthistory_hurdle_lgb conflict_ln
pr_46_cm_fatalities002_conflicthistory_hurdle_lgb_test.parquet
 .    4 fatalities002_conflicthistory_long_xgb conflictlong_ln
pr_46_cm_fatalities002_conflicthistory_long_xgb_test.parquet
 .    5 fatalities002_vdem_hurdle_xgb vdem_short
pr_46_cm_fatalities002_vdem_hurdle_xgb_test.parquet
 .    6 fatalities002_wdi_rf wdi_short
pr_46_cm_fatalities002_wdi_rf_test.parquet
 .    7 fatalities002_topics_rf topics_002
pr_46_cm_fatalities002_topics_rf_test.parquet
 .    8 fatalities002_topics_xgb topics_002
pr_46_cm_fatalities002_topics_xgb_test.parquet
 .    9 fatalities002_topics_hurdle_lgb topics_002
pr_46_cm_fatalities002_topics_hurdle_lgb_test.parquet
 .    10 fatalities002_joint_broad_rf joint_broad
pr_46_cm_fatalities002

In [ ]:
evaluation_allmodels

In [ ]:
ColsToShow = ['MSE_all','MSE_lowconflict',
              'MSE_all_fatalities002_topics_rf','MSE_lowconflict_fatalities002_topics_rf',
              'MSE_all_fatalities002_topics_hurdle_lgb','MSE_lowconflict_fatalities002_topics_hurdle_lgb',
             'MSE_all_fat_topics_rf','MSE_lowconflict_fat_topics_rf',]

evaluation_allmodels[ColsToShow]

In [ ]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels(level)
    
i = 0
for model in ModelList:
    print(i, model['modelname'], model['data_train'])
    i = i + 1
ModelList[8]['Evaluation_results_df']